In [4]:
#Pkg.add("PyCall")
using PyCall

In [5]:

@pyimport larlib

# Visualization by Giulio

In [6]:
function lar2plasm(EV, FE)
    EV = sparse(EV)
    faces = Array{Array{Int64, 1}, 1}()

    for f in 1:size(FE, 1)

        verts = Array{Int64, 1}()
        done = Array{Int64, 1}()
        face = FE[f, :]
        edges = EV[face.nzind, :]

        push!(done, 1)
        vs = edges[1, :].nzind
        if face.nzval[1] < 0
            vs = vs[end:-1:1]
        end
        startv, nextv = vs

        println(startv)
        push!(verts, startv)
        while nextv != startv
            e = setdiff(edges[:, nextv].nzind, done)[1]
            push!(done, e)
            vs = edges[e, :].nzind
            if face.nzval[e] < 0
                vs = vs[end:-1:1]
            end
            curv, nextv = vs
            push!(verts, curv)
        end

        push!(faces, verts)
    end

    "FV = " * string(faces)[15:end]
end

function lar2py(V, EV)
    text = "V = [[0,0]"
    for i in 1:size(V, 1)
        text = text * "," * string(vec(V[i, :]))
    end
    text = text * "]\n"
    
    text = text * "EV = ["
    for i in 1:size(EV, 1)
        if i > 1 text = text * "," end
        text = text * string(EV[i, :].nzind)
    end
    text = text * "]"
    return text
end


function visualize_numbers(set, font_size)
    """
    seg = [V, EV, FE]
    """

    f = open("tmp.py", "w")

    file_content = "from larlib import *\n" *
        lar2py(set[1], set[2]) * "\n" *
        lar2plasm(set[2], set[3]) * "\n" *
        "VV = AA(LIST)(range(len(V)))\n" *
        "submodel = STRUCT(MKPOLS((V,EV)))\n" *
        "VIEW(larModelNumbering(1,1,1)(V,[VV,EV,FV],submodel,$font_size))\n"
    print(file_content)
    write(f, file_content)

    close(f)
#     run(`python tmp.py`)
end

function visualize_boundary(set, boundary, font_size)
    f = open("tmp.py", "w")
    file_content =  "from larlib import *\n" *
        lar2py(set[1], set[2]) * "\n" *
        lar2plasm(set[2], set[3]) * "\n" *
        "VV = AA(LIST)(range(len(V)))\n" *
        "submodel = STRUCT(MKPOLS((V,EV)))\n" *
        "model = larModelNumbering(1,1,1)(V,[VV,EV,FV],submodel,$font_size)\n" *
        lar2py(set[1], set[2][boundary.nzind, :]) * "\n" *
        "overmodel = STRUCT(MKPOLS((V,EV)))\n" *
        "VIEW(STRUCT([model, COLOR(RED)(overmodel)]))"
    print(file_content)
    write(f, file_content)

    close(f)
    run(`python tmp.py`)
end

visualize_boundary (generic function with 1 method)

## My test object

In [7]:
V = [
    1 1
    2 1
    1 2
    2 2
    ]

4×2 Array{Int64,2}:
 1  1
 2  1
 1  2
 2  2

In [8]:
EV = sparse([
    1 -1 0 0
    0 1 -1 0
    -1 0 1 0
])

3×4 sparse matrix with 6 Int64 nonzero entries:
	[1, 1]  =  1
	[3, 1]  =  -1
	[1, 2]  =  -1
	[2, 2]  =  1
	[2, 3]  =  -1
	[3, 3]  =  1

In [9]:
FE = sparse([
        1 1 -1
        -1 -1 1
])

2×3 sparse matrix with 6 Int64 nonzero entries:
	[1, 1]  =  1
	[2, 1]  =  -1
	[1, 2]  =  1
	[2, 2]  =  -1
	[1, 3]  =  -1
	[2, 3]  =  1

In [10]:
set=[V, EV, FE]

3-element Array{AbstractArray{Int64,2},1}:
 [1 1; 2 1; 1 2; 2 2]                                                                   
 
	[1, 1]  =  1
	[3, 1]  =  -1
	[1, 2]  =  -1
	[2, 2]  =  1
	[2, 3]  =  -1
	[3, 3]  =  1
 
	[1, 1]  =  1
	[2, 1]  =  -1
	[1, 2]  =  1
	[2, 2]  =  -1
	[1, 3]  =  -1
	[2, 3]  =  1

# Rewrite into Julia with PyCall

In [11]:
V

4×2 Array{Int64,2}:
 1  1
 2  1
 1  2
 2  2

In [12]:
EV

3×4 sparse matrix with 6 Int64 nonzero entries:
	[1, 1]  =  1
	[3, 1]  =  -1
	[1, 2]  =  -1
	[2, 2]  =  1
	[2, 3]  =  -1
	[3, 3]  =  1

In [13]:
function ev2evpl(EV)
    EVpl = Array{Array{Int64, 1}, 1}()
    EV[1,:].nzind
    for i in 1:size(EV, 1)
        push!(EVpl, EV[i, :].nzind)
    end
    return  EVpl
end

ev2evpl (generic function with 1 method)

In [14]:
function v2vpl(V)
    Vpl = cat(1, zeros(V[1, :]'), V )
    return Vpl
end

v2vpl (generic function with 1 method)

In [15]:

function lar2plasm_jl(EV, FE)
    EV = sparse(EV)
    faces = Array{Array{Int64, 1}, 1}()

    for f in 1:size(FE, 1)

        verts = Array{Int64, 1}()
        done = Array{Int64, 1}()
        face = FE[f, :]
        edges = EV[face.nzind, :]

        push!(done, 1)
        vs = edges[1, :].nzind
        if face.nzval[1] < 0
            vs = vs[end:-1:1]
        end
        startv, nextv = vs

        println(startv)
        push!(verts, startv)
        while nextv != startv
            e = setdiff(edges[:, nextv].nzind, done)[1]
            push!(done, e)
            vs = edges[e, :].nzind
            if face.nzval[e] < 0
                vs = vs[end:-1:1]
            end
            curv, nextv = vs
            push!(verts, curv)
        end

        push!(faces, verts)
    end
    return faces
end


lar2plasm_jl (generic function with 1 method)

In [16]:
fontsize = 2
Vpl = v2vpl(V)
EVpl = ev2evpl(EV)
FVpl = lar2plasm_jl(EV, FE)
println("Vpl = ", Vpl)
println("EVpl = ", EVpl)
println("FVpl = ", FVpl)
inlist = collect(range(0, size(Vpl,1)))
println("inlist = ", inlist)
VV = larlib.AA(larlib.LIST)(inlist)
VV = [[0], [1], [2], [3], [4]]
submodel = larlib.STRUCT(larlib.MKPOLS((Vpl, EVpl)))
larlib.VIEW(larlib.larModelNumbering(1,1,1)(Vpl, [VV, EVpl, FVpl], submodel, fontsize))
println("VV = ", VV)


1
2
Vpl = [0 0; 1 1; 2 1; 1 2; 2 2]
EVpl = Array{Int64,1}[[1,2],[2,3],[1,3]]
FVpl = Array{Int64,1}[[1,2,3],[2,1,3]]
inlist = [0,1,2,3,4]
centre
centre
centre
Building batches from Hpc....
...done in 0 msec
Optimizing the octree....
   Number of input batches 73
   total number vertices    46
   Number of output batches 23
   Batch vertex media       2
...done in 0 msec
Building octree from 23 batches....
Scene number of nodes of the octree 45
Scene max depth                     4
Scene number of batches             23
...done in 0 msec
VV = Array{Int64,1}[[0],[1],[2],[3],[4]]


In [68]:

Vpl = [5 5; 1 1; 2 1; 1 2; 2 2]
EVpl = [1 2; 2 3; 1 3]
FVpl = [1 2 3; 2 1 3]

Vpl = [5 5; 1 1; 2 1; 1 2; 1 1]
EVpl = [ 1 2; 2 3; 1 3]
FVpl = [1 2 3; 2 1 3]
println("Vpl = ", Vpl)
println("EVpl = ", EVpl)
println("FVpl = ", FVpl)
inlist = collect(range(0, size(Vpl,1)))
println("inlist = ", inlist)
VV = larlib.AA(larlib.LIST)(inlist)
VV = [[0], [1], [2], [3], [4]]
submodel = larlib.STRUCT(larlib.MKPOLS((Vpl, EVpl)))
larlib.VIEW(larlib.larModelNumbering(1,1,1)(Vpl, [VV, EVpl, FVpl], submodel, fontsize))
println("VV = ", VV)
println(larlib.larModelNumbering(1,1,1)(Vpl, [VV, EVpl, FVpl], submodel, fontsize))
println("submodel = ", submodel)

Vpl = [5 5; 1 1; 2 1; 1 2; 1 1]
EVpl = [1 2; 2 3; 1 3]
FVpl = [1 2 3; 2 1 3]
inlist = [0,1,2,3,4]
centre
centre
centre
Building batches from Hpc....
...done in 1 msec
Optimizing the octree....
   Number of input batches 73
   total number vertices    24
   Number of output batches 12
   Batch vertex media       2
...done in 0 msec
Building octree from 12 batches....
Scene number of nodes of the octree 25
Scene max depth                     4
Scene number of batches             12
...done in 0 msec
VV = Array{Int64,1}[[0],[1],[2],[3],[4]]
PyObject <pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x7fba142c6870> >
submodel = PyObject <pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x7fba142c6720> >
centre
centre
centre


# Check identity with gulio code 

In [45]:
VV

5×1 Array{Any,2}:
 0
 1
 2
 3
 4

In [15]:
zeros(V[1,:]')

1×2 Array{Int64,2}:
 0  0

In [21]:

uu = lar2plasm(set[2], set[3])
println("output")
println(uu)

1
2
output
FV = [[1,2,3],[2,1,3]]


In [22]:

FVpl = lar2plasm_jl(set[2], set[3])
println("output")
println(FVpl)

1
2
output
Array{Int64,1}[[1,2,3],[2,1,3]]


In [25]:
function visualize_numbers_jl(set, font_size)
    """
    seg = [V, EV, FE]
    """
    @pyimport larlib
    FV


    file_content = "from larlib import *\n" *
        lar2py(set[1], set[2]) * "\n" *
        lar2plasm(set[2], set[3]) * "\n" *
        "VV = AA(LIST)(range(len(V)))\n" *
        "submodel = STRUCT(MKPOLS((V,EV)))\n" *
        "VIEW(larModelNumbering(1,1,1)(V,[VV,EV,FV],submodel,$font_size))\n"
    print(file_content)
    write(f, file_content)

    close(f)
#     run(`python tmp.py`)
end

visualize_numbers_jl (generic function with 1 method)

In [19]:
ev2evpl(EV)


3-element Array{Array{Int64,1},1}:
 [1,2]
 [2,3]
 [1,3]

In [17]:
set=[V, EV, FE]
lar2py(set[1], set[2]) 

"V = [[0,0],[1,1],[2,1],[1,2],[2,2]]\nEV = [[1,2],[2,3],[1,3]]"

In [18]:
v2vpl(V)

5×2 Array{Int64,2}:
 0  0
 1  1
 2  1
 1  2
 2  2